# CNN Trainig Templete
## Based on Tensorlfow - CNN MNIST example
https://github.com/tensorflow/tensorflow/blob/r1.6/tensorflow/examples/tutorials/layers/cnn_mnist.py

## 필요한 library 호출

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib.layers.python.layers import layers as layers_lib
from tensorflow.python.ops import array_ops

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

tf.logging.set_verbosity(tf.logging.INFO)

# GPU selection (memory)
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="3"


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data_txt = open('./data_split/5fold_0802.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [3]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 224
r_h = 224

total_pix = r_w * r_h * channel

In [4]:
tr_data_dir['train0'][0]

'/dataset_full/Torn/Torn (1220).png'

In [5]:
def vgg16(features, labels, mode):
    
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, height, width, channels]
    # Our Fishing net image size is 640x480 and 3-channel (RGB)
    input_layer = tf.reshape(features["x"], [-1, 224, 224, 3])
    dropout_keep_prob = 0.5
    num_classes = 2
    is_training = True
    
    
    net = layers_lib.repeat(
        input_layer, 2, layers.conv2d, 64, [3, 3], scope='conv1')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool1')
    net = layers_lib.repeat(net, 2, layers.conv2d, 128, [3, 3], scope='conv2')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool2')
    net = layers_lib.repeat(net, 3, layers.conv2d, 256, [3, 3], scope='conv3')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool3')
    net = layers_lib.repeat(net, 3, layers.conv2d, 512, [3, 3], scope='conv4')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool4')
    net = layers_lib.repeat(net, 3, layers.conv2d, 512, [3, 3], scope='conv5')
    net = layers_lib.max_pool2d(net, [2, 2], scope='pool5')
    # Use conv2d instead of fully_connected layers.
    net = layers.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
    net = layers_lib.dropout(
        net, dropout_keep_prob, is_training=is_training, scope='dropout6')
    net = layers.conv2d(net, 4096, [1, 1], scope='fc7')
    net = layers_lib.dropout(
        net, dropout_keep_prob, is_training=is_training, scope='dropout7')
    net = layers.conv2d(
        net,
        num_classes, [1, 1],
        activation_fn=None,
        normalizer_fn=None,
        scope='fc8')
    net = array_ops.squeeze(net, [1, 2], name='fc8/squeezed')
    
    logits = net
    
    with tf.device('/GPU:3'):

        predictions = {
            # Generate predictions (for PREDICT and EVAL mode)
            "classes": tf.argmax(input=logits, axis=1),
            # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
            # `logging_hook`.
            "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
        }
        if mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

        # Calculate Loss (for both TRAIN and EVAL modes)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
        # sparse_softmax_cross_entropy cannot use one-hot encoding

        #loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)

        # Configure the Training Op (for TRAIN mode)
        if mode == tf.estimator.ModeKeys.TRAIN:
            #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
            optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
            train_op = optimizer.minimize(
                loss=loss,
                global_step=tf.train.get_global_step())
            return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

        # Add evaluation metrics (for EVAL mode)
        eval_metric_ops = {
            "accuracy": tf.metrics.accuracy(
                labels=labels, predictions=predictions["classes"])}
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [8]:
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.9
#session = tf.Session(config=config)

k = 1
total_res = {}

for i in range(k):
    val_result = []

    f_dir = "/models/ICROS_vgg/kfold2_%d/" % i

    if not tf.gfile.Exists(f_dir):
        tf.gfile.MakeDirs(f_dir)

    net_classifier = tf.estimator.Estimator(
        model_fn=vgg16, model_dir=f_dir)

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)
    
    
    
    tr_data = np.zeros((len(tr_data_dir['train%d'%i]), total_pix), dtype=np.float32)
    tr_label = np.zeros((len(tr_data_dir['train%d'%i]), 1), dtype=np.int32)

    for j in range(len(tr_data)):
        img = cv2.imread(tr_data_dir['train%d'%i][j])
        img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
        tr_data[j,:] = img2.flatten()

        if 'Untorn' in tr_data_dir['train%d'%i][j]:
            tr_label[j] = 0
        elif 'normal' in tr_data_dir['train%d'%i][j]:
            tr_label[j] = 0

        else:
            tr_label[j] = 1

    
    val_data = np.zeros((len(tr_data_dir['valid%d'%i]), total_pix), dtype=np.float32)
    val_label = np.zeros((len(tr_data_dir['valid%d'%i]), 1), dtype=np.int32)

    for j in range(len(val_data)):
        img = cv2.imread(tr_data_dir['valid%d'%i][j])
        img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
        val_data[j,:] = img2.flatten()

        if 'Untorn' in tr_data_dir['valid%d'%i][j]:
            val_label[j] = 0
        elif 'normal' in tr_data_dir['valid%d'%i][j]:
            val_label[j] = 0

        else:
            val_label[j] = 1
            
            
            
            
    batch_s = 32
    epochs = 30
    tr_steps = int(tr_data.shape[0]/batch_s*epochs)
    step_in_epoch = int(tr_data.shape[0]/batch_s)+1
    in_steps = 1


    for ee in range(epochs):
        
        train_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": tr_data},
            y=tr_label,
            batch_size=batch_s,
            num_epochs=1,
            shuffle=False)

        eval_input_fn = tf.estimator.inputs.numpy_input_fn(
            x={"x": val_data},
            y=val_label,
            batch_size=64,
            num_epochs=1,
            shuffle=False)

        net_classifier.train(
            input_fn=train_input_fn,
            steps=step_in_epoch,
            hooks=[logging_hook])

        eval_results = net_classifier.evaluate(input_fn=eval_input_fn, hooks=[logging_hook])
        temp_res = eval_results["accuracy"]

        #total_res['%d'%i][ee] = temp_res

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_service': None, '_evaluation_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efef85f82b0>, '_tf_random_seed': None, '_session_config': None, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_is_chief': True, '_global_id_in_cluster': 0, '_save_checkpoints_steps': None, '_model_dir': '/models/ICROS_vgg/kfold2_0/', '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_task_type': 'worker', '_save_checkpoints_secs': 600}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/ICROS_vgg/kfold2_0/model.ckpt-2300
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints

INFO:tensorflow:step = 2516, loss = 3.8153652e-05 (32.107 sec)
INFO:tensorflow:Saving checkpoints for 2530 into /models/ICROS_vgg/kfold2_0/model.ckpt.
INFO:tensorflow:Loss for final step: 3.9736427e-08.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-02-06:32:19
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/ICROS_vgg/kfold2_0/model.ckpt-2530
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:probabilities = [[0.9999987  0.00000128]
 [0.00001307 0.9999869 ]
 [1.         0.        ]
 [1.         0.        ]
 [0.9999999  0.00000014]
 [0.00000003 1.        ]
 [0.9995554  0.00044462]
 [1.         0.        ]
 [0.00000003 1.        ]
 [0.9999713  0.00002868]
 [0.         1.        ]
 [1.         0.00000003]
 [0.9989611  0.00103892]
 [1.         0.        ]
 [0.         1.        ]
 [0.         1.        ]
 [1.         0.        ]
 [0

INFO:tensorflow:step = 2646, loss = 0.0003777175
INFO:tensorflow:probabilities = [[0.         1.        ]
 [1.         0.        ]
 [0.00000001 1.        ]
 [0.99910694 0.00089301]
 [0.00001559 0.9999844 ]
 [0.         1.        ]
 [0.15083946 0.84916055]
 [1.         0.        ]
 [0.0000256  0.99997437]
 [0.         1.        ]
 [0.00000004 1.        ]
 [1.         0.        ]
 [0.         1.        ]
 [1.         0.        ]
 [0.00000008 0.9999999 ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [0.00000031 0.99999964]
 [0.00002554 0.9999745 ]
 [1.         0.        ]
 [0.         1.        ]
 [1.         0.        ]
 [0.         1.        ]
 [0.9999999  0.00000008]
 [0.         1.        ]
 [0.9999882  0.00001185]
 [0.         1.        ]
 [1.         0.        ]
 [0.         1.        ]
 [0.9998042  0.0001958 ]
 [0.00000006 1.        ]] (16.253 sec)
INFO:tensorflow:global_step/sec: 3.08873
INFO:tensorflow:probabilities = [[0.00000037 0.99999964]
 [0.999

INFO:tensorflow:Finished evaluation at 2018-08-02-06:34:42
INFO:tensorflow:Saving dict for global step 2875: accuracy = 0.99451756, global_step = 2875, loss = 0.089398995
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/ICROS_vgg/kfold2_0/model.ckpt-2875
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2876 into /models/ICROS_vgg/kfold2_0/model.ckpt.
INFO:tensorflow:probabilities = [[0.0162674  0.98373264]
 [0.00000636 0.9999937 ]
 [0.         1.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [0.00000164 0.99999833]
 [1.         0.00000005]
 [1.         0.        ]
 [0.00000043 0.9999995 ]
 [1.         0.        ]
 [0.00001951 0.99998045]
 [1.         0.        ]
 [1.         0.        ]
 [1.        

INFO:tensorflow:step = 3091, loss = 2.0078844e-06 (32.385 sec)
INFO:tensorflow:Saving checkpoints for 3105 into /models/ICROS_vgg/kfold2_0/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-02-06:36:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /models/ICROS_vgg/kfold2_0/model.ckpt-3105
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:probabilities = [[0.9999999  0.00000016]
 [0.00001751 0.9999825 ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.00000001]
 [0.00000014 0.9999999 ]
 [0.9999999  0.00000012]
 [1.         0.        ]
 [0.         1.        ]
 [0.9999958  0.00000414]
 [0.         1.        ]
 [1.         0.        ]
 [0.99995685 0.00004313]
 [1.         0.        ]
 [0.         1.        ]
 [0.         1.        ]
 [1.         0.        ]
 [1.         

INFO:tensorflow:step = 3221, loss = 2.5968988e-05
INFO:tensorflow:probabilities = [[0.         1.        ]
 [1.         0.        ]
 [0.00000008 0.9999999 ]
 [0.9999502  0.0000498 ]
 [0.00000956 0.99999046]
 [0.         1.        ]
 [0.11046359 0.88953644]
 [1.         0.        ]
 [0.00002112 0.9999789 ]
 [0.         1.        ]
 [0.00000027 0.99999976]
 [1.         0.        ]
 [0.         1.        ]
 [1.         0.        ]
 [0.00000007 0.9999999 ]
 [1.         0.        ]
 [1.         0.        ]
 [1.         0.        ]
 [0.00000004 1.        ]
 [0.00000199 0.999998  ]
 [1.         0.        ]
 [0.         1.        ]
 [1.         0.        ]
 [0.         1.        ]
 [1.         0.        ]
 [0.         1.        ]
 [1.         0.        ]
 [0.         1.        ]
 [1.         0.        ]
 [0.         1.        ]
 [0.9999765  0.00002346]
 [0.00000003 1.        ]] (16.330 sec)
INFO:tensorflow:global_step/sec: 3.07454
INFO:tensorflow:probabilities = [[0.00000361 0.9999964 ]
 [1.  

INFO:tensorflow:Finished evaluation at 2018-08-02-06:38:33
INFO:tensorflow:Saving dict for global step 3450: accuracy = 0.97587717, global_step = 3450, loss = 0.09048795
